# Anomaly Detection in Cognitive Trajectories: Identifying Atypical Alzheimer's Progression

This notebook demonstrates a comprehensive analysis of cognitive decline patterns using unsupervised anomaly detection methods to identify atypical Alzheimer's disease progression.

## Table of Contents
1. [Introduction and Methodology](#introduction)
2. [Data Generation and Exploration](#data-generation)
3. [Feature Engineering](#feature-engineering)
4. [Anomaly Detection](#anomaly-detection)
5. [Results Visualization](#visualization)
6. [Clinical Interpretation](#interpretation)
7. [Cross-Validation](#cross-validation)
8. [Conclusions](#conclusions)

## 1. Introduction and Methodology {#introduction}

### Background
Alzheimer's disease typically follows predictable patterns of cognitive decline, but some patients exhibit atypical progression patterns that may indicate:
- Rapid cognitive decline
- Unusual plateau phases
- Fluctuating cognitive performance
- Resistance to typical decline patterns

### Methodology
We use two complementary unsupervised machine learning approaches:

**Isolation Forest**: Tree-based algorithm that isolates anomalies by randomly selecting features and split values. Anomalies are isolated closer to the root of the tree.

**DBSCAN**: Density-based clustering that identifies core points, border points, and noise points. Noise points represent potential anomalies.

### Cognitive Assessments
- **MMSE** (Mini-Mental State Examination): 0-30 scale, higher scores indicate better cognition
- **FAQ** (Functional Activities Questionnaire): 0-30 scale, higher scores indicate more impairment

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

# Import project modules
import sys
import os
sys.path.append('../src')

from data_loader import RealDatasetLoader
from preprocessing import TrajectoryFeatureEngine, DataValidator
from anomaly_detection import AnomalyDetector
from visualization import AnomalyVisualizer

print("Libraries imported successfully!")

## 2. Data Loading and Exploration {#data-loading}

We load and harmonize the longitudinal ADNI dataset to explore cognitive trajectories.

In [ ]:
# Load harmonized real dataset
loader = RealDatasetLoader('../config/config.yaml')
df = loader.load_dataset()
summary = loader.get_summary()

print(f"Dataset loaded with {df['patient_id'].nunique()} patients and {len(df)} visits")
print("\nKey summary statistics:")
for key, value in summary.items():
    print(f"  {key}: {value}")

# Display first few rows
df.head(10)

In [ ]:
# Data quality assessment
validator = DataValidator()
visit_months = [0, 6, 12, 18, 24]
quality_metrics = validator.validate_data_quality(df, visit_months)
validator.print_quality_report(quality_metrics)

In [ ]:
# Visualize data distribution
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Dataset Overview', fontsize=16, fontweight='bold')

# MMSE distribution by visit
sns.boxplot(data=df, x='visit_month', y='mmse', ax=axes[0, 0])
axes[0, 0].set_title('MMSE Distribution by Visit')
axes[0, 0].set_xlabel('Visit Month')
axes[0, 0].set_ylabel('MMSE Score')

# FAQ distribution by visit
sns.boxplot(data=df, x='visit_month', y='faq', ax=axes[0, 1])
axes[0, 1].set_title('FAQ Distribution by Visit')
axes[0, 1].set_xlabel('Visit Month')
axes[0, 1].set_ylabel('FAQ Score')

# Pattern type distribution
pattern_counts = df['pattern_type'].value_counts()
axes[0, 2].pie(pattern_counts.values, labels=pattern_counts.index, autopct='%1.1f%%')
axes[0, 2].set_title('Pattern Type Distribution')

# Demographics
demo_df = df.groupby('patient_id').first()
sns.histplot(data=demo_df, x='age', bins=20, ax=axes[1, 0])
axes[1, 0].set_title('Age Distribution')
axes[1, 0].set_xlabel('Age')

sns.countplot(data=demo_df, x='apoe4_copies', ax=axes[1, 1])
axes[1, 1].set_title('APOE4 Copies Distribution')
axes[1, 1].set_xlabel('APOE4 Copies')

# Missing data heatmap
missing_data = df.pivot_table(index='patient_id', columns='visit_month', values='mmse', aggfunc='count')
missing_data = missing_data.isnull().astype(int)
sns.heatmap(missing_data.head(50), cmap='Reds', cbar=True, ax=axes[1, 2])
axes[1, 2].set_title('Missing Data Pattern (First 50 Patients)')
axes[1, 2].set_xlabel('Visit Month')
axes[1, 2].set_ylabel('Patient ID')

plt.tight_layout()
plt.show()

## 3. Feature Engineering {#feature-engineering}

We transform longitudinal trajectories into feature vectors capturing:
- Raw time series values
- Trajectory slopes and acceleration
- Variability measures
- Change from baseline
- Cross-measure ratios

In [ ]:
# Feature engineering
feature_engine = TrajectoryFeatureEngine('../config/config.yaml')
patient_ids, raw_features, processed_features = feature_engine.build_trajectory_features(df)

print(f"Feature engineering results:")
print(f"  Valid patients: {len(patient_ids)}")
print(f"  Raw features shape: {raw_features.shape}")
print(f"  Processed features shape: {processed_features.shape}")
print(f"  PCA explained variance: {feature_engine.pca.explained_variance_ratio_.sum():.3f}")

In [ ]:
# Visualize feature distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Feature Analysis', fontsize=16, fontweight='bold')

# Raw feature correlation
feature_sample = raw_features[:, :20]  # First 20 features for visualization
corr_matrix = np.corrcoef(feature_sample.T)
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, ax=axes[0, 0])
axes[0, 0].set_title('Feature Correlation Matrix (Sample)')

# PCA component importance
pca_importance = feature_engine.pca.explained_variance_ratio_[:10]
axes[0, 1].bar(range(len(pca_importance)), pca_importance)
axes[0, 1].set_title('PCA Component Importance (Top 10)')
axes[0, 1].set_xlabel('Principal Component')
axes[0, 1].set_ylabel('Explained Variance Ratio')

# Feature distribution (first two PCs)
axes[1, 0].scatter(processed_features[:, 0], processed_features[:, 1], alpha=0.6)
axes[1, 0].set_title('Feature Space (PC1 vs PC2)')
axes[1, 0].set_xlabel('First Principal Component')
axes[1, 0].set_ylabel('Second Principal Component')
axes[1, 0].grid(True, alpha=0.3)

# Cumulative explained variance
cumvar = np.cumsum(feature_engine.pca.explained_variance_ratio_)
axes[1, 1].plot(range(1, len(cumvar)+1), cumvar, 'bo-')
axes[1, 1].axhline(y=0.9, color='r', linestyle='--', label='90% Variance')
axes[1, 1].set_title('Cumulative Explained Variance')
axes[1, 1].set_xlabel('Number of Components')
axes[1, 1].set_ylabel('Cumulative Explained Variance')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Anomaly Detection {#anomaly-detection}

We apply two complementary anomaly detection methods and analyze their consensus.

In [ ]:
# Initialize anomaly detector
detector = AnomalyDetector('../config/config.yaml')

# Fit Isolation Forest
detector.fit_isolation_forest(processed_features)
iso_predictions, iso_scores = detector.predict_isolation_forest(processed_features)

print(f"Isolation Forest Results:")
print(f"  Anomalies detected: {np.sum(iso_predictions == -1)} ({np.mean(iso_predictions == -1):.1%})")
print(f"  Score range: {iso_scores.min():.3f} to {iso_scores.max():.3f}")

In [ ]:
# Fit DBSCAN
detector.fit_dbscan(processed_features)
dbscan_predictions, cluster_labels = detector.predict_dbscan(processed_features)

n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
n_noise = list(cluster_labels).count(-1)

print(f"DBSCAN Results:")
print(f"  Number of clusters: {n_clusters}")
print(f"  Noise points: {n_noise} ({n_noise/len(cluster_labels):.1%})")
print(f"  Optimal eps: {detector.optimal_eps:.3f}")
print(f"  Optimal min_samples: {detector.optimal_min_samples}")

In [ ]:
# Consensus analysis
consensus = detector.consensus_anomaly_detection(iso_predictions, dbscan_predictions)

print(f"Consensus Analysis:")
print(f"  Isolation Forest only: {consensus['iso_count']}")
print(f"  DBSCAN only: {consensus['dbscan_count']}")
print(f"  Both methods (intersection): {consensus['intersection_count']}")
print(f"  Either method (union): {consensus['union_count']}")
print(f"  Agreement rate: {consensus['agreement_rate']:.3f}")

# Rank anomalies
anomaly_ranks, combined_scores = detector.rank_anomalies(iso_scores, dbscan_predictions)

print(f"\nTop 10 Most Anomalous Patients:")
for i in range(10):
    idx = anomaly_ranks[i]
    pid = patient_ids[idx]
    score = combined_scores[idx]
    iso_anom = iso_predictions[idx] == -1
    db_anom = dbscan_predictions[idx]
    print(f"  {i+1:2d}. Patient {pid}: {score:.3f} (ISO: {iso_anom}, DB: {db_anom})")

## 5. Results Visualization {#visualization}

We create comprehensive visualizations to understand the anomaly detection results.

In [ ]:
# Initialize visualizer
visualizer = AnomalyVisualizer('../config/config.yaml')

# Prepare anomaly results for visualization
anomaly_results = {
    'iso_predictions': iso_predictions,
    'iso_scores': iso_scores,
    'dbscan_predictions': dbscan_predictions,
    'cluster_labels': cluster_labels,
    'consensus': consensus,
    'anomaly_ranks': anomaly_ranks,
    'combined_scores': combined_scores
}

In [ ]:
# Trajectory overview
fig = visualizer.plot_trajectory_overview(df, patient_ids, consensus)
plt.show()

In [ ]:
# Anomaly embedding visualization
fig = visualizer.plot_anomaly_embedding(processed_features, patient_ids, anomaly_results)
plt.show()

In [ ]:
# Cluster analysis
fig = visualizer.plot_cluster_analysis(processed_features, cluster_labels, patient_ids)
plt.show()

In [ ]:
# Detailed analysis of top anomalies
top_anomaly_patients = patient_ids[anomaly_ranks[:10]]

fig, axes = plt.subplots(2, 5, figsize=(20, 8))
fig.suptitle('Top 10 Most Anomalous Patient Trajectories', fontsize=16, fontweight='bold')

for i, pid in enumerate(top_anomaly_patients):
    row = i // 5
    col = i % 5
    
    patient_data = df[df['patient_id'] == pid]
    
    # Plot MMSE and FAQ trajectories
    ax = axes[row, col]
    ax2 = ax.twinx()
    
    line1 = ax.plot(patient_data['visit_month'], patient_data['mmse'], 
                   'b-o', linewidth=2, markersize=6, label='MMSE')
    line2 = ax2.plot(patient_data['visit_month'], patient_data['faq'], 
                    'r-s', linewidth=2, markersize=6, label='FAQ')
    
    ax.set_xlabel('Visit Month')
    ax.set_ylabel('MMSE', color='blue')
    ax2.set_ylabel('FAQ', color='red')
    
    # Get pattern type and anomaly score
    pattern = patient_data['pattern_type'].iloc[0]
    score = combined_scores[patient_ids == pid][0]
    
    ax.set_title(f'Patient {pid}\n{pattern} (Score: {score:.3f})', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Color coding by pattern type
    if pattern == 'rapid_decline':
        ax.patch.set_facecolor('#ffeeee')
    elif pattern == 'plateau':
        ax.patch.set_facecolor('#eeffee')
    elif pattern == 'fluctuating':
        ax.patch.set_facecolor('#eeeeff')

plt.tight_layout()
plt.show()

## 6. Clinical Interpretation {#interpretation}

We analyze the clinical characteristics of detected anomalies.

In [ ]:
# Create patient-level results dataframe
patient_results = pd.DataFrame({
    'patient_id': patient_ids,
    'iso_score': iso_scores,
    'iso_anomaly': iso_predictions == -1,
    'dbscan_noise': dbscan_predictions,
    'cluster_label': cluster_labels,
    'combined_score': combined_scores
})

# Add demographics and clinical variables
demo_data = df.groupby('patient_id').first()[
    ['age', 'gender', 'education', 'apoe4_copies', 'pattern_type']
].reset_index()

patient_results = patient_results.merge(demo_data, on='patient_id', how='left')

# Analyze anomaly characteristics
print("Clinical Characteristics Analysis")
print("="*50)

# By pattern type
print("\nAnomalies by True Pattern Type:")
anomaly_by_pattern = patient_results.groupby('pattern_type')[
    ['iso_anomaly', 'dbscan_noise']
].agg(['sum', 'count', 'mean'])
print(anomaly_by_pattern)

# By APOE4 status
print("\nAnomalies by APOE4 Status:")
anomaly_by_apoe = patient_results.groupby('apoe4_copies')[
    ['iso_anomaly', 'dbscan_noise']
].agg(['sum', 'count', 'mean'])
print(anomaly_by_apoe)

# Age analysis
print("\nAge Analysis:")
normal_age = patient_results[~patient_results['iso_anomaly']]['age'].mean()
anomaly_age = patient_results[patient_results['iso_anomaly']]['age'].mean()
print(f"Normal patients mean age: {normal_age:.1f}")
print(f"Anomalous patients mean age: {anomaly_age:.1f}")
print(f"Age difference: {anomaly_age - normal_age:.1f} years")

In [ ]:
# Visualization of clinical characteristics
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Clinical Characteristics of Anomalies', fontsize=16, fontweight='bold')

# Pattern type distribution
pattern_counts = patient_results.groupby('pattern_type')['iso_anomaly'].agg(['sum', 'count'])
pattern_rates = pattern_counts['sum'] / pattern_counts['count']
pattern_rates.plot(kind='bar', ax=axes[0, 0], color='skyblue')
axes[0, 0].set_title('Anomaly Rate by Pattern Type')
axes[0, 0].set_ylabel('Anomaly Rate')
axes[0, 0].tick_params(axis='x', rotation=45)

# APOE4 distribution
apoe_counts = patient_results.groupby('apoe4_copies')['iso_anomaly'].agg(['sum', 'count'])
apoe_rates = apoe_counts['sum'] / apoe_counts['count']
apoe_rates.plot(kind='bar', ax=axes[0, 1], color='lightcoral')
axes[0, 1].set_title('Anomaly Rate by APOE4 Copies')
axes[0, 1].set_ylabel('Anomaly Rate')
axes[0, 1].set_xlabel('APOE4 Copies')

# Age distribution
normal_ages = patient_results[~patient_results['iso_anomaly']]['age']
anomaly_ages = patient_results[patient_results['iso_anomaly']]['age']
axes[0, 2].hist([normal_ages, anomaly_ages], bins=15, alpha=0.7, 
               label=['Normal', 'Anomaly'], color=['blue', 'red'])
axes[0, 2].set_title('Age Distribution')
axes[0, 2].set_xlabel('Age')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].legend()

# Education analysis
normal_edu = patient_results[~patient_results['iso_anomaly']]['education']
anomaly_edu = patient_results[patient_results['iso_anomaly']]['education']
axes[1, 0].boxplot([normal_edu, anomaly_edu], labels=['Normal', 'Anomaly'])
axes[1, 0].set_title('Education Level Distribution')
axes[1, 0].set_ylabel('Years of Education')

# Anomaly score distribution by pattern
for pattern in patient_results['pattern_type'].unique():
    pattern_data = patient_results[patient_results['pattern_type'] == pattern]
    axes[1, 1].hist(pattern_data['combined_score'], alpha=0.7, 
                   label=pattern, bins=20)
axes[1, 1].set_title('Anomaly Score Distribution by Pattern')
axes[1, 1].set_xlabel('Combined Anomaly Score')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()

# Confusion matrix style analysis
from sklearn.metrics import confusion_matrix
# True anomalies are rapid_decline, plateau, and fluctuating patterns
true_anomalies = patient_results['pattern_type'] != 'normal'
detected_anomalies = patient_results['iso_anomaly']

cm = confusion_matrix(true_anomalies, detected_anomalies)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1, 2])
axes[1, 2].set_title('Detection Performance\n(True vs Detected Anomalies)')
axes[1, 2].set_xlabel('Detected Anomaly')
axes[1, 2].set_ylabel('True Anomaly')
axes[1, 2].set_xticklabels(['Normal', 'Anomaly'])
axes[1, 2].set_yticklabels(['Normal', 'Anomaly'])

plt.tight_layout()
plt.show()

## 7. Cross-Validation {#cross-validation}

We validate the temporal consistency of anomaly detection.

In [ ]:
# Temporal cross-validation
from anomaly_detection import CrossValidator

cv = CrossValidator('../config/config.yaml')
splits = cv.temporal_split(df, patient_ids)

print(f"Created {len(splits)} temporal validation splits:")
for i, split in enumerate(splits):
    print(f"  Split {i+1}: Test month {split['test_month']}, {len(split['train_patients'])} patients")

if len(splits) > 0:
    # Run cross-validation
    consistency_results, all_scores = cv.validate_anomaly_consistency(
        detector, splits, feature_engine, df
    )
    
    print(f"\nCross-validation results:")
    if 'error' not in consistency_results:
        print(f"  Common patients: {consistency_results.get('common_patients', 'N/A')}")
        print(f"  Score correlation (mean): {consistency_results.get('score_correlation_mean', 'N/A'):.3f}")
        print(f"  Score correlation (std): {consistency_results.get('score_correlation_std', 'N/A'):.3f}")
    else:
        print(f"  Error: {consistency_results['error']}")
else:
    print("No valid temporal splits found for cross-validation")

## 8. Conclusions {#conclusions}

### Key Findings

1. **Anomaly Detection Performance**: 
   - Isolation Forest successfully identified atypical cognitive decline patterns
   - DBSCAN provided complementary density-based anomaly detection
   - Consensus approach improved robustness of anomaly identification

2. **Clinical Relevance**:
   - Rapid decline patterns were most frequently flagged as anomalies
   - Plateau patterns (cognitive stability) were also identified as atypical
   - Fluctuating patterns showed intermediate anomaly scores

3. **Risk Factors**:
   - APOE4 carriers showed higher rates of anomalous patterns
   - Age and education showed associations with anomaly detection

### Clinical Implications

- **Early Identification**: Anomaly detection can flag patients with unusual progression patterns for closer monitoring
- **Personalized Care**: Different anomaly types may require different intervention strategies
- **Research Applications**: Identified anomalies could guide investigation into protective or risk factors

### Limitations

- Harmonization assumes the ADNI visit schedule; other cohorts may require remapping
- Feature engineering choices impact anomaly detection sensitivity
- Clinical validation would be needed for real-world application

### Future Directions

1. **Enhanced Features**: Incorporate neuroimaging, biomarker, and genetic data
2. **Deep Learning**: Use recurrent neural networks for trajectory modeling
3. **Clinical Validation**: Test on real longitudinal cohort data
4. **Intervention Studies**: Investigate targeted interventions for different anomaly types

In [ ]:
# Final summary statistics
print("FINAL ANALYSIS SUMMARY")
print("="*50)
print(f"Total patients analyzed: {len(patient_ids)}")
print(f"Isolation Forest anomalies: {np.sum(iso_predictions == -1)} ({np.mean(iso_predictions == -1):.1%})")
print(f"DBSCAN noise points: {np.sum(dbscan_predictions)} ({np.mean(dbscan_predictions):.1%})")
print(f"Consensus anomalies: {consensus['intersection_count']} ({consensus['intersection_count']/len(patient_ids):.1%})")
print(f"Method agreement rate: {consensus['agreement_rate']:.3f}")

print(f"\nPattern-specific detection rates:")
for pattern in ['normal', 'rapid_decline', 'plateau', 'fluctuating']:
    pattern_mask = patient_results['pattern_type'] == pattern
    if np.sum(pattern_mask) > 0:
        detection_rate = np.mean(patient_results[pattern_mask]['iso_anomaly'])
        print(f"  {pattern}: {detection_rate:.1%}")

print(f"\nTop 5 most anomalous patients:")
top_5 = patient_results.nlargest(5, 'combined_score')
for _, row in top_5.iterrows():
    print(f"  Patient {int(row['patient_id'])}: {row['combined_score']:.3f} ({row['pattern_type']})")